In [ ]:
import subprocess
smi=subprocess.run(["nvidia-smi", "-L"], stdout=subprocess.PIPE)
if smi.returncode != 0:
  assert False, "No GPU available!"
print(smi.stdout.decode())

GPU 0: Tesla T4 (UUID: GPU-41ed80c3-5105-8f43-1108-9ed46036dd2c)



<a href="https://colab.research.google.com/github/mildinvestor/katago-colab/blob/master/colab_katago_gd_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This document shows how to run KataGo on Colab, and how to connect it using `Sabaki`, `Lizzie` (or other GTP engine supported apps) in your local machine.

# Step 1: Run KataGo and SSH server on Colab Server

Before running, you should prepare these 2 values: `NGROK_TOKEN` and `USER_PASSWORD`. 

* NGROK_TOKEN - You can register or login in `ngrok` website here: [https://dashboard.ngrok.com/auth/your-authtoken](https://dashboard.ngrok.com/auth/your-authtoken). Then you can find your own ngrok token.
* USER_PASSWORD - You can use any password as you wish. (should only contains letters or digits)

`USER_PASSWORD` will be used in `Sabaki` or `Lizzie` as engine options. (More details will be elaborated in Step 2 below).

Change these 2 values of your own in the parameter from the right parameter entry field(`Config for SSH`). 

In addition, change the `Config for KataGo` as needed.

* WEIGHT_FILE - You can choose the number of blocks in your deep learning network file. If you set it to "AUTO", it will be automatically set according to the performance of the GPU assigned to Google Colab.
* rules - You can choose the rules of Go.
* maxVisits - You can enter the maximum number of visits to the node. The larger this number, the more time KataGo will take to search the game tree during a game(in genmove mode).

Then click the `Run` button. 

After run, do the following:
* Authenticate your Google Drive. Please access the authentication URL displayed, log in to Google, and then allow authentication. Enter the verification code displayed in the `Enter verification code:` field.
* Copy the value of `SSH_INFO_GOOGLE_DRIVE_FILE_ID` displayed in the output cell. This will be used as engine options. This value will be the same after the next execution.

It may take about 2~3 minutes to complete running the code. 

In [ ]:
# SSH configuration
NGROK_TOKEN = 'your_ngrok_token'
USER_PASSWORD = 'your_password'

# KataGo configuration
KATAGO_BACKEND = 'AUTO'
T4_WEIGHT = '40b'
ELSE_WEIGHT = '20b'

# Constant
KATAGO_COLAB_REPOSITORY_URL = 'https://github.com/matobataketoshi/katago-colab.git'
KATAGO_CONFIG_FILE = '/content/katago-colab/config/gtp_colab.cfg'
KATAGO_TUNING_DIR = '/content/katago-colab/opencltuning'
KATAGO_RESOURCE_FILE = '/content/katago-colab/colab-resource/external-resource.json'
SSH_INFO_FILE_NAME = 'colab-katago-ssh.json'
SSH_INFO_DIR = '/content/drive/MyDrive'

# Install useful stuff
!echo "Install libraries"
!apt-get update 1>/dev/null
!apt-get install --yes ssh screen nano htop ranger git libzip4 1>/dev/null
!wget -q https://github.com/wonsiks/katago-colab/releases/download/v1.9.1/libzip.so.5.0 -O /usr/lib/x86_64-linux-gnu/libzip.so.5
!pip install -U -q PyDrive 1>/dev/null

import subprocess
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import requests
import json
import os
from re import sub

def get_katago_resource(katago_resource_file):
    with open(katago_resource_file, 'r') as f:
        resource = json.load(f)
    return resource

def save_ssh_info(dir, file_name):
    file_path = dir + '/' + file_name
    ssh_option = create_ssh_option()
    file_id = get_file_id_in_drive(file_path)
    if file_id is None:
        file_id = upload_ssh_option_with_pydrive(file_name, ssh_option)
    else:
        ssh_option['fileId'] = file_id
        write_ssh_option(file_path, ssh_option)
    print('================================================================')
    print('SSH_INFO_GOOGLE_DRIVE_FILE_ID: {}'.format(file_id))
    print('================================================================')
    return file_id

def create_ssh_option():
    ssh_option = None
    try:
        r = requests.get('http://localhost:4040/api/tunnels')
        raw_ssh = r.json()['tunnels'][0]['public_url']
        ssh_args = (sub("tcp://", "", raw_ssh)).split(':')
        ssh_option = {
            'host': ssh_args[0],
            'port': int(ssh_args[1]),
            'user': 'root'
        }
    except Exception as e:
        print('Failed to create SSH options. Make sure that the ngrok token is set correctly.')
        raise e
    return ssh_option

def get_file_id_in_drive(file_path):
    file_id = None
    if (os.path.exists(file_path)):
        try:
            with open(file_path, 'r') as f:
                ssh_option = json.load(f)
                file_id = ssh_option.get('fileId')
        except Exception as e:
            file_id = None
    return file_id

def write_ssh_option(file_path, ssh_option):
    print('Write SSH option: {}'.format(file_path))
    with open(file_path, 'w') as f:
        f.write(json.dumps(ssh_option))
 
def upload_ssh_option_with_pydrive(file_name, ssh_option):
    print('Authenticate to Google Drive')
    drive = auth_google_drive()
    file_metadata = {
        'title': file_name,
        'mimeType': 'application/json'
    }
    file_list = drive.ListFile({'q': 'title="' + file_name + '" and trashed=False'}).GetList()
    if (len(file_list) > 0):
        file_metadata['id'] = file_list[0]['id']
    ssh_option_file = drive.CreateFile(file_metadata)
    ssh_option_file.Upload()
    file_id = ssh_option_file.get('id')
    ssh_option['fileId'] = file_id
    ssh_option_file.SetContentString(json.dumps(ssh_option))
    ssh_option_file.Upload()
    ssh_option_file.InsertPermission({'type': 'anyone', 'value': 'anyone', 'role': 'reader'})
    return file_id

def auth_google_drive():
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    return drive

# Get GPU name
gpu_name = str(subprocess.check_output("nvidia-smi -q | grep \"Product Name\" | cut -d\":\" -f2 | tr -cd '[:alnum:]._-'", shell=True), encoding='utf-8')
print('GPU: {}'.format(gpu_name))

# Select backend type
if KATAGO_BACKEND == "AUTO":
  if gpu_name == "TeslaT4":
    KATAGO_BACKEND = "TensorRT"
  else:
    KATAGO_BACKEND = "OPENCL"    
print('Using KataGo Binery: {}'.format(KATAGO_BACKEND))

# Select weight file
if gpu_name == "TeslaT4":
  WEIGHT_FILE = T4_WEIGHT
else:
  WEIGHT_FILE = ELSE_WEIGHT
print('Using KataGo Weight: {}'.format(WEIGHT_FILE))

%cd /content
# Clone katago-colab
!echo "Git clone katago-colab"
!rm -rf katago-colab
!git clone $KATAGO_COLAB_REPOSITORY_URL 1>/dev/null

# Get URLs of external resources
katago_resource = get_katago_resource(KATAGO_RESOURCE_FILE)
ngrok_url = katago_resource['ngrok']
katago_url = katago_resource[KATAGO_BACKEND]
weight_url = katago_resource[WEIGHT_FILE]

if KATAGO_BACKEND == "TensorRT":
  !echo "Install the files needed to drive TensorRT."

  %cd /content
  !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/libcudnn8_8.2.1.32-1+cuda11.3_amd64.deb
  !dpkg -i libcudnn8_8.2.1.32-1+cuda11.3_amd64.deb

  !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/libnvinfer8_8.2.2-1+cuda11.4_amd64.deb
  !dpkg -i libnvinfer8_8.2.2-1+cuda11.4_amd64.deb
  !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/libnvonnxparsers8_8.2.2-1+cuda11.4_amd64.deb
  !dpkg -i libnvonnxparsers8_8.2.2-1+cuda11.4_amd64.deb
  !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/libnvparsers8_8.2.2-1+cuda11.4_amd64.deb
  !dpkg -i libnvparsers8_8.2.2-1+cuda11.4_amd64.deb

# Download ngrok
!wget -q $ngrok_url -O ngrok
!chmod +x /content/ngrok
 
# SSH setting
!echo "root:$USER_PASSWORD" | chpasswd
!echo "PasswordAuthentication yes" > /etc/ssh/sshd_config
!echo "PermitUserEnvironment yes" >> /etc/ssh/sshd_config
!echo "PermitRootLogin yes" >> /etc/ssh/sshd_config

!mkdir -p /root/.ssh
!service ssh restart > /dev/null
 
# Run ngrok
!echo "Run ngrok."
get_ipython().system_raw('./ngrok authtoken $NGROK_TOKEN && ./ngrok tcp 22 &')
!sleep 5

# Save SSH info to Google Drive
!echo "Save SSH connection information in Google Drive."
SSH_INFO_GOOGLE_DRIVE_FILE_ID = save_ssh_info(SSH_INFO_DIR, SSH_INFO_FILE_NAME)

# Download KataGo binary
!echo "Receive the Katago executable file."
!wget -q "$katago_url" -O katago
!chmod +x /content/katago
!/content/katago version

# Put KataGo tuning files
!mkdir -p /root/.katago/
if KATAGO_BACKEND == "OPENCL":
  !cp -r $KATAGO_TUNING_DIR /root/.katago/

# Download a network file of KataGo
!echo "Download a network file of KataGo"
!rm -rf weight.bin.gz
!wget -q "$weight_url" -O "weight.bin.gz"

# Copy enviroment variable for SSH
with open('/root/.ssh/environment','w') as f:
    for key, value in os.environ.items():
        print('{}={}'.format(key, value), file=f)

!echo -e "\n[KataGo Config]" 
!cat $KATAGO_CONFIG_FILE
!echo -e "\n"

!echo "Ready to connect! You can connect lizzie ..."
!echo "lizzie engine drive command : ./colab-katago-gd.exe $SSH_INFO_GOOGLE_DRIVE_FILE_ID $USER_PASSWORD"

#Step 2: Connect Colab KataGo via Sabaki or Lizzie

## 1) first, download a colab-katago client app
Here are the download Links:  

**For Windows Users (64bit windows)**  
https://github.com/wonsiks/katago-colab/releases/download/v1.9.1/colab-katago-gd.windows.zip

https://github.com/wonsiks/katago-colab/releases/download/v1.9.1/colab-katago-gd.exe

**For Linux Users**  
https://github.com/wonsiks/katago-colab/releases/download/v1.9.1/colab-katago-gd.linux.zip

**For Mac OSX Users**  
https://github.com/wonsiks/katago-colab/releases/download/v1.9.1/colab-katago-gd.mac.zip

**after download completed, unzip it, you can get a binary program naming  colab-katago-gd or colab-katago-gd.exe**

## 2) then, add engine in Sabaki or Lizzie

To configure the engine in `Sabaki` or `Lizzie`, you just need to fill the absolute path of your `colab-katago` program (which you've downloaded just now), and the file ID, password(i.e, `SSH_INFO_GOOGLE_DRIVE_FILE_ID` displayed in the output cell, `USER_PASSWORD` you configured in Step 1).

```
<Absolute path of colab-katago-gd> <SSH_INFO_GOOGLE_DRIVE_FILE_ID> <USER_PASSWORD>
```

**Sabaki Example**:  

![Sabaki Example Image](
https://mildinvestor.com/wp-content/uploads/2021/08/colab-katago-sabaki-example.jpg
)  

**Lizzie Example**:  

![Lizzie Example Image](https://mildinvestor.com/wp-content/uploads/2021/08/colab-katago-lizzie-example.jpg
)

## More Config Options (Optional)
You can use the following engine options to limits the KataGo Search `visits` or `time` (in seconds) in genmove mode. For example: 

```
<AbsolutePathOfColabKataGoProglem> <SSH_INFO_GOOGLE_DRIVE_FILE_ID> <USER_PASSWORD> 30s
```
The above options `30s` limits the search time for each move to 30 seconds.
```
<AbsolutePathOfColabKataGoProglem> <SSH_INFO_GOOGLE_DRIVE_FILE_ID> <USER_PASSWORD> 1600v
```
The above options `1600v` limits the search visits for each move to 1600 visits.

You can change the numbers 30 or 1600 in the above to any number as you want.

#The following sections are for debugging only, you can ignore.

**Shows the Colab GPU Info**

In [ ]:
!nvidia-smi

Get your ssh login info
*ssh login account is root, the login password is the `USER_PASSWORD` you configured in the previous steps*

In [ ]:
import requests
from re import sub
r = requests.get('http://localhost:4040/api/tunnels')
str_ssh = r.json()['tunnels'][0]['public_url']
str_ssh = sub("tcp://", "", str_ssh)
str_ssh = sub(":", " -p ", str_ssh)
str_ssh = "ssh root@" + str_ssh 
print(str_ssh)

**Restart SSH**


In [ ]:
!service ssh restart